# [Explorations of Action - MoA EDA](https://www.kaggle.com/headsortails/explorations-of-action-moa-eda)

## 1 Introduction

これは、きちんとしたRとggplot2とのメカニズムの作用（MoA）競争のための包括的な探索的データ分析です。それはあなたが始めるために必要なすべての情報を提供するはずです、そして多分少しでも。

この課題の目的は、「生物活性に基づいて薬物を分類する」ことです。医薬品の発見は、特定の疾患に関連する特定のタンパク質を特定し、それらのタンパク質を標的とする分子を開発することを目的としています。分子のMoAはその生物活性をコード化します。私たちのデータセットは、さまざまな薬物に対する100種類のヒト細胞の反応を記述しています。これらの応答パターンは、MoA応答を分類するために使用されます。

これはマルチラベル分類の問題です。薬物には、さまざまな方法でさまざまな細胞型からのバイナリ応答を記述する複数のMoAアノテーションを含めることができます。評価指標は、列ごとの平均ログ損失です。

データは、慣れ親しんだトレーニングおよびテストファイルの形で提供されます。他のコンテストとは対照的に、ここにはトレーニング予測子（train_features.csv）とターゲット（train_targets_scored.csv）の2つの別々のファイルがあります。各行は特定の処理に対応しています。さらに、予測する必要はないがコンテキスト分析に使用できるオプションのMoAターゲットセット（train_targets_nonscored.csv）も提供されます。

具体的な目標は、テストファイルtest_features.csvのtrain_targets_scored.csvクラスの確率を予測することです。パブリックリーダーボードはこのテストデータの約25％に基づいており、75％はプライベートLBに割り当てられていることに注意してください。

始めましょう！

## 3 Overview: File structure and content

まず、データセットとその形状の概要を簡単に説明します。 以下の表はインタラクティブで、最初の50行のすべての列をカバーしています。

Training dataset  
我々は気づく：

- これは、約900列のかなり広いデータセットです。 データの説明から、「g-」で始まる機能が遺伝子発現データをエンコードし（そのうち772あります）、「c-」で始まる機能（合計100）が細胞生存率データを示すことがわかります。

- さらに、3つの「cp_」機能があります。cp_typeはサンプルの処理を開始し、cp_timeとcp_doseは処理の期間と投与量をエンコードします。

- sig_idは、サンプルの一意の主キーです。

#### Test dataset  
我々は気づく：

テストデータにはトレーニングデータと同じ機能が含まれており、トレーニングデータの24k行と比較して約4k行があります。 200サンプルと4000サンプルの比率はそれほど高くありません。

#### Targets  
我々は気づく：

ターゲットの応答もかなり広いです。 200を超える異なるバイナリ出力を備えています。 予想通り、このテーブルはかなりまばらです。

sig_idは、予測子をターゲットに結合するために使用できます。

#### Non-scored targets  
我々は気づく：

追加のスコアなしターゲットには、約400のクラス（列）が含まれます。 得点したターゲットの2倍。

それは、薬物自体に関するかなり多くの追加情報です。 良いアプリケーションが見つかるかどうか見てみましょう。

#### Missing values  
No missing values in either train or test data:

#### Target sparsity  
ターゲットクラスが非常に多いため、MoAはまれである可能性があります。 ターゲットのデータフレームはかなりまばらになります。 ここに番号があります：

「ゼロ以外のターゲットクラス値の割合：0.343％」  
ゼロ以外の半分未満。 非常に不均衡です。 スコアなしのターゲットはどうですか？

「ゼロ以外のスコアなしのターゲットクラス値の割合：0.052％」  
ワオ; それは大きな違いです。 すでに散在しているスコアリングターゲットよりも、ほぼ1桁も散在しています。

#### Quality checks  
まず、簡単な品質チェックで、sig_idの値が本当に一意であることを確認します。 結果はゼロになるはずです：

次に、trainとtargetのsig_id値が一致するかどうかを確認する別のチェックを行います。 結果もゼロになるはずです：

すべてが期待どおりに見えます。 とても良い。

## 4 Individual feature visualisations
まず、さまざまな予測機能とターゲット機能の分布を個別にプロットしてから、マルチ機能のビジュアルと相関に進みます。 ここでは、グループごとに機能を扱います。

### 4.1 Treatment features
これらは、サンプルがどのように処理されたかを、用量、期間の観点からより一般的に説明する予測機能です。 そしてそれが「本当の」治療であったのか、それとも対照であったのか。

### 4.2 Gene expression features
これらは本質的に匿名の機能であり、「g-0」から「g-771」までラベルが付けられています。 それらの値は数値なので、例として最初の4つの遺伝子の特徴の密度を見てみましょう。

### 4.3 Cell viability features
遺伝子の特徴と同様に、細胞生存率の特徴は匿名で、「c-0」から「c-99」までラベル付けされています。 100の機能。 それらの分布は次のようになります。

### 4.4 Targets
すべてのターゲットはバイナリ列で、特定の細胞タイプが薬剤に反応するかどうかを示します。 一部のターゲットクラスは、応答のタイプも測定します。 例えば adrenergic_receptor_agonistクラスとadrenergic_receptor_antagonistクラスがあります。 それらはおそらく同じサンプル行に対してアクティブであってはなりません。

私たちの課題はマルチラベル分類の問題であり、そのため、行（つまり、薬物サンプル）には複数のMoAが含まれる可能性があります（つまり、複数のターゲットクラスがアクティブになる可能性があります）。 まず、一度にアクティブにできるターゲットクラスの数の分布を見てみましょう。

ここでは、新しいdplyr行ごとの構文と哲学を使用していることに注意してください。これにより、標準のdplyr動詞を行に簡単に適用できます。 コードセルを展開して、動作を確認します。


我々は気づく：

- トレーニングサンプルの約39％にMoAアノテーションがまったくありません（たとえば、すべてのターゲットクラスの値がゼロです）。 40％が完全に空である場合、これはまばらなターゲットデータフレームを説明する特定の方法になります。

- サンプルの50％を少し超える最大のグループには、正確に1つのMoA注釈があります（たとえば、1つのクラス=その行の「1」の1つの値）。

- 2つ以上のMoAアノテーションについて、7つの同時MoA（0.03％のケース）まで下がる尾が表示されます。 2 MoAの場合のみが5％をわずかに上回り、3 MoAsの場合は1％をわずかに上回ります。 他のすべてのケースは非常にまれです。 6 MoAのインスタンスはないことに注意してください。

ターゲットを別の角度から見る：トレーニングデータでMoAのインスタンスが最も多いクラスはどれですか？ つまり ゼロの中で1が最も多い列はどれですか？ このため、すべての列を合計するだけで済みます。 結果の分布をプロットし、トップクラスとボトムクラスを確認します。 密度プロットの対数のx軸に注意してください。

我々は気づく：

- ほとんどのクラスでは、24k行のトレーニングサンプル内に約10〜200のMoAがあります。 破線は、40でのピークを示します。アクティブな行の最大数は800をわずかに上回り、最小数は1です。

- 右下のパネルの2つの暗赤色のケースについて、MoAが1つだけのまれなケースが測定されます。 上位のスコアリングクラスは、左上のパネルに表示されます。 両極端は主に阻害剤と拮抗剤を特徴としています。

しばしば、この最後の観察は別の調査の道を開きます。 クラスの名前は私のような素人にはほとんど言いませんが、「阻害剤」、「拮抗薬」、「作動薬」などがかなりあることに気づかざるを得ません。 それらを抽出して、それらの頻度を見てみましょう。 すべてのクラス名がこれらのグループに適切に分類されるわけではないため、2回以上出現するクラス名のみを調べます。 アイデアは、クラスの名前を使用して、その最後の単語を抽出することです。 結果の周波数は次のとおりです。

我々は気づく：

- クラス名には非常に多くの共通点があります。50％以上が「阻害剤」、「拮抗薬」、「作動薬」はそれぞれ約15％の同等レベルであり、残りの3つの用語ははるかにまれです。

- その他すべて、クラスの約12％には、これらの単純なパターンに当てはまらない名前があります。

## 5 Multiple feature interaction visuals
個々の機能がどのように動作するかについての理解が深まったので、それらの相互作用を調べてみましょう。 最初に、同じセット内のさまざまな機能を確認してから、分析をそれらのグループとその構成要素間の相互作用に拡張します。 上記と同じ順序で機能セットを使用します。